**Universidad de los Andes**<br>
Facultad de Ingeniería<br>
__Maestría en Inteligencia Artificial__

# Alcanzando los objetivos de desarrollo sostenible.

Autores: __Andrés Vega__ y __David Méndez Acuña__

### Introducción

In [4]:
import pandas as pd
from datetime import datetime

from sklearn.base import BaseEstimator, TransformerMixin

In [5]:
class Context:
    '''
    Clase que permite almacenar el contexto de ejecución de los pasos 
    del pipeline. Esta clase será particularmente útil en el último 
    paso del pipeline donde necesitaremoslos datos generados por todos 
    los pasos del pipeline para imprimir el detalle de los resultados.
    '''
    def __init__(self, model) -> None:
        self.model = model

    def log(message=None):
        '''
        Imprime el mensaje que entra por parámetro acompañado de la hora 
        actual. Este método será util para la trazabilidad de la ejecución
        del pipeline.  
        '''
        now = datetime.now()
        print(f'[INFO] {now.strftime("%H:%M:%S")} — {message}')

In [ ]:
class FileLoader(BaseEstimator, TransformerMixin) :
    '''
    Este paso del pipeline se encarga de cargar el archivo con los datos de  
    entrenamiento a partir de un archivo. En este paso se recibe la ruta del 
    archivo y se retorna un data frame con los datos organizados de manera tabular. 
    '''
    def __init__(self, context) -> None :
        self.context = context

    def fit(self, X, y=None) :
        return self
    
    def transform(self, X, y=None):
        '''
        Recibe la ruta de un archivo en el parámetro X y retorna un data frame
        con los datos organizados de manera tabular. 
        
        Parámetros relevantes
        ------------------------
        X : str
            La ruta del archivo que contiene los datos de entrenamiento. 
        '''
